# DynamoDB

## Intalar DynamoDB-local docker

```bash
docker run -d \
  -p 8000:8000 \
  -v $(pwd)/dynamodb_data:/home/dynamodblocal/data \
  amazon/dynamodb-local \
  -jar DynamoDBLocal.jar -sharedDb -dbPath /home/dynamodblocal/data
```

script

```bash
#!/bin/bash

# Nombre del contenedor
CONTENEDOR="dynamodb-local"

# Ruta del volumen de datos
DATA_DIR="$HOME/dynamodb_data"

# Crear carpeta de datos si no existe
mkdir -p "$DATA_DIR"

# Verificar si el contenedor ya existe
if docker ps -a --format '{{.Names}}' | grep -q "^${CONTENEDOR}$"; then
    echo "El contenedor '$CONTENEDOR' ya existe."

    # Si no está corriendo, iniciarlo
    if [ "$(docker inspect -f '{{.State.Running}}' $CONTENEDOR)" != "true" ]; then
        echo "Iniciando el contenedor..."
        docker start $CONTENEDOR
    else
        echo "Ya está corriendo."
    fi
else
    echo "Creando e iniciando el contenedor '$CONTENEDOR'..."
    docker run -d \
        --name $CONTENEDOR \
        -p 8000:8000 \
        -v "$DATA_DIR":/home/dynamodblocal/data \
        amazon/dynamodb-local \
        -jar DynamoDBLocal.jar -sharedDb -dbPath /home/dynamodblocal/data
fi

# Mostrar logs recientes
echo "Logs recientes:"
docker logs --tail 5 $CONTENEDOR
```

In [41]:
import boto3

## Crear un cliente de boto para dynamodb-local

In [2]:
cliente = boto3.client(
        "dynamodb",
        endpoint_url="http://localhost:8000",  # 🔸 DynamoDB Local
        region_name="us-west-2",               # 🔹 Requerido por boto3 (puede ser cualquiera)
        aws_access_key_id="fakeMyKeyId",       # 🔹 Cualquier valor
        aws_secret_access_key="fakeSecretKey"  # 🔹 Cualquier valor
    )

In [4]:
cliente.list_tables()  # 🔸 Listar tablas

{'TableNames': [],
 'ResponseMetadata': {'RequestId': '4687e77d-4c30-47f6-a321-2d24ffe0186d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Jetty(12.0.14)',
   'date': 'Sat, 31 May 2025 18:15:04 GMT',
   'x-amzn-requestid': '4687e77d-4c30-47f6-a321-2d24ffe0186d',
   'content-type': 'application/x-amz-json-1.0',
   'x-amz-crc32': '1315925753',
   'content-length': '17'},
  'RetryAttempts': 0}}

## Crear una tabla

In [12]:
def create_table_db(table_name: str, **kwargs):
    """
    Crea una tabla en DynamoDB usando un cliente de boto3 previamente configurado.

    Parámetros:
    -----------
    table_name : str
        El nombre de la tabla que se quiere crear.
    
    **kwargs :
        Otros parámetros necesarios para definir la tabla, como:
            - AttributeDefinitions
            - KeySchema
            - ProvisionedThroughput
            - GlobalSecondaryIndexes (opcional)
            - LocalSecondaryIndexes (opcional)

    Retorna:
    --------
    dict
        La respuesta del cliente de boto3 con la descripción de la tabla creada.
    
    Nota:
    -----
    Esta función espera que el objeto `client` ya esté creado en el ámbito global.
    Además, utiliza un "waiter" de boto3 para esperar a que la tabla esté disponible antes de continuar.
    """

    # Llamamos a la función create_table del cliente para crear la tabla con los parámetros recibidos.
    response = cliente.create_table(TableName=table_name, **kwargs)

    # Obtenemos un "esperador" (waiter) que se bloquea hasta que la tabla exista (es decir, esté lista).
    waiter = cliente.get_waiter("table_exists")
    waiter.wait(TableName=table_name)

    # Devolvemos la respuesta que contiene los detalles de la tabla recién creada.
    return response

In [8]:
COURSE_PREFIX = "Data-Engineering"

In [9]:
capacity_units = {'ReadCapacityUnits': 10, 'WriteCapacityUnits': 5}


🧱 ¿Qué representa esta estructura?

```python 
capacity_units = {'ReadCapacityUnits': 10, 'WriteCapacityUnits': 5}
```

Esto define la capacidad provisionada de la tabla (solo se usa en modo provisionado, incluso si estás usando DynamoDB local):

- ReadCapacityUnits: número de unidades de lectura por segundo.
- WriteCapacityUnits: número de unidades de escritura por segundo.

En DynamoDB local no afecta al rendimiento, pero sigue siendo obligatorio.

In [10]:

product_catalog_table = {'table_name': f'{COURSE_PREFIX}-ProductCatalog',
                         'kwargs': {
                             'KeySchema': [{'AttributeName': 'Id', 'KeyType': 'HASH'}],
                             'AttributeDefinitions': [{'AttributeName': 'Id', 'AttributeType': 'N'}],
                             'ProvisionedThroughput': capacity_units
                         }
                        }

- KeySchema

  ```python
  'KeySchema': [{'AttributeName': 'Id', 'KeyType': 'HASH'}] 
  ```

  - Define la clave primaria de la tabla.
    - Aquí se usa una clave de partición (HASH) llamada "Id".
    - No se usa clave de ordenación (RANGE), por lo tanto, esta tabla no está compuesta.


- AttributeDefinitions

  ```python
  'AttributeDefinitions': [{'AttributeName': 'Id', 'AttributeType': 'N'}]
  ```

  - Se especifica el tipo de atributo para las claves.
    - "AttributeType": "N" indica que el campo Id es un número (Number).
    - Otros posibles valores son "S" (String) o "B" (Binary).

- ProvisionedThroughput

  ```python
  'ProvisionedThroughput': capacity_units
  ```

  - Usa el diccionario capacity_units definido anteriormente.
    - Esto se pasa tal cual a la API de boto3.

In [13]:
response = create_table_db(table_name=product_catalog_table['table_name'], **product_catalog_table["kwargs"]) 
# print(response)

In [14]:
import json

In [15]:
print(json.dumps(response, indent=2, default=str))  # 🔸 Imprimir la respuesta de la creación de la tabla

{
  "TableDescription": {
    "AttributeDefinitions": [
      {
        "AttributeName": "Id",
        "AttributeType": "N"
      }
    ],
    "TableName": "Data-Engineering-ProductCatalog",
    "KeySchema": [
      {
        "AttributeName": "Id",
        "KeyType": "HASH"
      }
    ],
    "TableStatus": "ACTIVE",
    "CreationDateTime": "2025-05-31 15:32:11.895000-03:00",
    "ProvisionedThroughput": {
      "LastIncreaseDateTime": "1969-12-31 21:00:00-03:00",
      "LastDecreaseDateTime": "1969-12-31 21:00:00-03:00",
      "NumberOfDecreasesToday": 0,
      "ReadCapacityUnits": 10,
      "WriteCapacityUnits": 5
    },
    "TableSizeBytes": 0,
    "ItemCount": 0,
    "TableArn": "arn:aws:dynamodb:ddblocal:000000000000:table/Data-Engineering-ProductCatalog",
    "DeletionProtectionEnabled": false
  },
  "ResponseMetadata": {
    "RequestId": "c2de7ac0-b9e7-4c63-8afc-066b6eac0d5f",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "server": "Jetty(12.0.14)",
      "date": "Sat, 3

In [17]:
def delete_table_db(table_name: str):
    """
    Elimina una tabla de DynamoDB usando un cliente de boto3 previamente configurado.

    Parámetros:
    -----------
    table_name : str
        El nombre de la tabla que se desea eliminar.

    Retorna:
    --------
    dict
        La respuesta del cliente de boto3, que contiene detalles del proceso de eliminación.
    
    Nota:
    -----
    Esta función asume que el objeto `cliente` ya está creado en el ámbito global,
    y configurado para conectarse a DynamoDB (ya sea en AWS o local).
    """

    # Llamamos a delete_table para eliminar la tabla con el nombre indicado.
    response = cliente.delete_table(TableName=table_name)

    return response


In [34]:
print(json.dumps(delete_table_db('Data-Engineering-ProductCatalog'), indent=2, default=str))

{
  "TableDescription": {
    "AttributeDefinitions": [
      {
        "AttributeName": "id",
        "AttributeType": "N"
      }
    ],
    "TableName": "Data-Engineering-ProductCatalog",
    "KeySchema": [
      {
        "AttributeName": "id",
        "KeyType": "HASH"
      }
    ],
    "TableStatus": "ACTIVE",
    "CreationDateTime": "2025-05-31 15:55:30.404000-03:00",
    "ProvisionedThroughput": {
      "LastIncreaseDateTime": "1969-12-31 21:00:00-03:00",
      "LastDecreaseDateTime": "1969-12-31 21:00:00-03:00",
      "NumberOfDecreasesToday": 0,
      "ReadCapacityUnits": 10,
      "WriteCapacityUnits": 5
    },
    "TableSizeBytes": 0,
    "ItemCount": 0,
    "TableArn": "arn:aws:dynamodb:ddblocal:000000000000:table/Data-Engineering-ProductCatalog",
    "DeletionProtectionEnabled": false
  },
  "ResponseMetadata": {
    "RequestId": "748ed7ec-610c-4aa4-b9d7-bdd46a107478",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "server": "Jetty(12.0.14)",
      "date": "Sat, 3

In [19]:
cliente.list_tables()  # 🔸 Listar tablas

{'TableNames': [],
 'ResponseMetadata': {'RequestId': '358408aa-03a4-4171-a6bb-ff26042af1ee',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Jetty(12.0.14)',
   'date': 'Sat, 31 May 2025 18:35:51 GMT',
   'x-amzn-requestid': '358408aa-03a4-4171-a6bb-ff26042af1ee',
   'content-type': 'application/x-amz-json-1.0',
   'x-amz-crc32': '1315925753',
   'content-length': '17'},
  'RetryAttempts': 0}}

___

In [23]:
for tabla in conn.tables.all():
    print(tabla.name)                    # 🔸 Imprimir nombre de cada tabla
    print(tabla.creation_date_time)      # 🔸 Imprimir fecha de creación de cada tabla
    print(tabla.item_count)              # 🔸 Imprimir cantidad de items en cada tabla
    print(tabla.table_status)            # 🔸 Imprimir estado de cada tabla
    print(tabla.table_arn)               # 🔸 Imprimir ARN de cada tabla
    print(tabla.provisioned_throughput)  # 🔸 Imprimir throughput provisionado
    print(tabla)                         # 🔸 Imprimir objeto tabla completo

🆚 client vs resource en boto3

| Aspecto                      | `client`                                          | `resource`                                                   |
| ---------------------------- | ------------------------------------------------- | ------------------------------------------------------------ |
| 🔧 **Nivel de abstracción**  | Bajo (más cerca de la API HTTP de AWS)            | Alto (más orientado a objetos, más fácil de usar)            |
| 🧱 **Estilo**                | Basado en funciones (`dict` de entrada/salida)    | Basado en objetos (`objeto.metodo()`)                        |
| 📦 **Datos devueltos**       | `dict` (típico JSON con estructura API de AWS)    | Objetos de Python con atributos y métodos                    |
| 🧭 **Validación/Control**    | Más preciso, ideal para configuraciones complejas | Más simple, ideal para tareas comunes                        |
| 📜 **Documentación oficial** | Reproduce 1:1 la API de AWS                       | Tiene métodos adicionales que no siempre se documentan igual |


🧠 ¿Cuál usar?

| Caso de uso                                  | Recomendación |
| -------------------------------------------- | ------------- |
| Operaciones simples y legibles               | ✅ `resource`  |
| Acceso completo a todas las funciones de API | ✅ `client`    |
| Control total sobre headers y parámetros     | ✅ `client`    |
| Rápida escritura de scripts CRUD             | ✅ `resource`  |


🎯 Tip final:

En muchos proyectos se usa resource para el código principal y client cuando se necesita algo más avanzado, como esperar a que una tabla exista (client.get_waiter()).

In [26]:
conn = boto3.resource(
    'dynamodb',
    region_name='us-west-2',
    endpoint_url='http://localhost:8000', # importante para conexión local
    aws_access_key_id='fakeMyKeyId',
    aws_secret_access_key='fakeSecretAccessKey')

In [27]:
conn.tables.all()

dynamodb.tablesCollection(dynamodb.ServiceResource(), dynamodb.Table)

Datos para crear la tabla

```python
product_catalog_table = {'table_name': f'{COURSE_PREFIX}-ProductCatalog',
                         'kwargs': {
                             'KeySchema': [{'AttributeName': 'Id', 'KeyType': 'HASH'}],
                             'AttributeDefinitions': [{'AttributeName': 'Id', 'AttributeType': 'N'}],
                             'ProvisionedThroughput': capacity_units
                         }
                        }
capacity_units = {'ReadCapacityUnits': 10, 'WriteCapacityUnits': 5}
```

In [38]:
try :
    conn.Table('Data-Engineering-ProductCatalog').load()
    print("la tabla existe") # 🔸 Verificar si la tabla existe
except Exception as e:
    print(e)  # 🔸 Imprimir el error si la tabla no existe
    print("la tabla no existe, se creará una nueva")
    tabla = conn.create_table(
    TableName =  "Data-Engineering-ProductCatalog",
    KeySchema = [
        {
            'AttributeName': 'id',               # Clave primaria
            'KeyType': 'HASH'                    # Puede ser HASH (partición) o RANGE (ordenación)
        }
            ],
    AttributeDefinitions = [
        {
            'AttributeName': 'id',               # Nombre del atributo
            'AttributeType': 'N'                 # Tipo de dato: N (número), S (cadena), B (binario)
        }
            ],
    ProvisionedThroughput = {
       'ReadCapacityUnits': 10,
       'WriteCapacityUnits': 5
        }
    )
    # Esperar a que la tabla esté disponible
    tabla.meta.client.get_waiter('table_exists').wait(TableName=tabla.name)
    print(f"Tabla creada: {tabla.table_status}")


la tabla existe


In [39]:
for tabla in conn.tables.all():
    print(tabla.name)                    # 🔸 Imprimir nombre de cada tabla
    print(tabla.creation_date_time)      # 🔸 Imprimir fecha de creación de cada tabla
    print(tabla.item_count)              # 🔸 Imprimir cantidad de items en cada tabla
    print(tabla.table_status)            # 🔸 Imprimir estado de cada tabla
    print(tabla.table_arn)               # 🔸 Imprimir ARN de cada tabla
    print(tabla.provisioned_throughput)  # 🔸 Imprimir throughput provisionado
    print(tabla)                         # 🔸 Imprimir objeto tabla completo

Data-Engineering-ProductCatalog
2025-05-31 16:07:47.761000-03:00
0
ACTIVE
arn:aws:dynamodb:ddblocal:000000000000:table/Data-Engineering-ProductCatalog
{'LastIncreaseDateTime': datetime.datetime(1969, 12, 31, 21, 0, tzinfo=tzlocal()), 'LastDecreaseDateTime': datetime.datetime(1969, 12, 31, 21, 0, tzinfo=tzlocal()), 'NumberOfDecreasesToday': 0, 'ReadCapacityUnits': 10, 'WriteCapacityUnits': 5}
dynamodb.Table(name='Data-Engineering-ProductCatalog')


___